# NMF machine learning model

### Import necessary packages 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

from keras import layers
from keras import losses
import keras

from keras.layers import TextVectorization

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import NMF

In [2]:
## Initializing numpy array to replicate our csv file

In [3]:
m = np.array([
    [0, 1, 0, 1, 2, 2, 0, 4] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 1, 1, 1, 3, 4, 1, 2] + [round(np.random.rand(),3) for _ in range(5)],
    [2, 3, 1, 1, 2, 2, 2, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [1, 1, 1, 0, 1, 1, 4, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 2, 3, 4, 1, 1, 2, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 0, 0, 0, 1, 0, 3, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 1, 0, 1, 2, 2, 4, 4] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 1, 0, 1, 2, 2, 3, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 1, 1, 1, 3, 4, 3, 4] + [round(np.random.rand(),3) for _ in range(5)],
    [2, 3, 1, 1, 2, 2, 4, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [1, 1, 1, 0, 1, 1, 3, 2] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 2, 3, 4, 1, 1, 2, 3] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 0, 0, 0, 1, 0, 1, 0] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 1, 0, 1, 2, 2, 3, 0] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 1, 0, 1, 2, 2, 0, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 1, 1, 1, 3, 4, 2, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [2, 3, 1, 1, 2, 2, 1, 4] + [round(np.random.rand(),3) for _ in range(5)],
    [1, 1, 1, 0, 1, 1, 3, 2] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 2, 3, 4, 1, 1, 4, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 0, 0, 0, 1, 0, 3, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 1, 0, 1, 2, 2, 0, 2] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 1, 0, 1, 2, 2, 1, 4] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 1, 1, 1, 3, 4, 0, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [2, 3, 1, 1, 2, 2, 3, 0] + [round(np.random.rand(),3) for _ in range(5)],
    [1, 1, 1, 0, 1, 1, 1, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 2, 3, 4, 1, 1, 3, 4] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 0, 0, 0, 1, 0, 4, 1] + [round(np.random.rand(),3) for _ in range(5)],
    [0, 1, 0, 1, 2, 2, 4, 3] + [round(np.random.rand(),3) for _ in range(5)]
])

for row in m:
    row[:8] = np.random.randint(0, 2, size=8)

# I've initialized the first half of the columns as 1's and 0's to mimic how the csv file will have 1's 
# for when the book can be found in that genre and 0's for when that book is not found in that genre
# the second half of the columns I've given random floats to mimic the percentages that we'll gather from 
# the open library reviews

In [4]:
# I've listed about 30 of the keywords from reviews

reviews = [
    'Fast paced', 'Medium paced', 'Slow paced', 'Meandering','Engaging', 'Exciting', 'Neutral', 
    'Boring', 'Dense', 'Clearly written', 'Succinct', 'Effective explanations', 
    'Well organized', 'Beginner', 'Intermediate', 'Advanced', 'Layman', 
    'Comprehensive', 'Focused', 'Horror', 'Romance', 'Mystery', 'Fiction', 'Drama', 'Fantasy', 'Humor', 
    'Review', 'Classic', 'Satire', 'Exploratory', 'Philisophical', 'Crime', 'Textbook', 'Suspenseful', 
    'Romantic', 'Emotional', 'Scientific', 'Tense', 'Hopeful',
    'Dark', 'Sad', 'Strange', 'Mysterious', 'Informative', 
    'Fearful', 'Angry', 'Highly recommend', 'Recommend', 
    'Life changing', 'Quotable', 'Underrated', 'Medium', 'Long', 'Short',
    'Trendy', 'Accurate', 'Biased', 'Adult themes', 
    'Trigger warnings', 'Offensive language', 'Olde', 'Slang', 
    'Entertainment', 'Broaden perspective', 'Inspiration']

## Begin creating the matrices V, W, H

In [5]:
# the columns of the csv will start with the genres/ subjects
# then I've added the first five of the keywords from reviews to create all of our columns

dataset = pd.DataFrame(m, columns = ["Romance", "Fantasy", "Historical Fiction", "Horror", "Humor", "Literature", 
                 "Mystery/Thriller", "Science Fiction"] + reviews[:5])

# the rows of the csv file will have book titles
dataset.index = [
    "The Great Gatsby", "To Kill a Mockingbird", "1984", "Pride and Prejudice", "Moby-Dick", "War and Peace", "The Catcher in the Rye",
    "The Hobbit", "Ulysses", "The Odyssey", "Crime and Punishment", "Jane Eyre", "Brave New World", "Wuthering Heights", "The Divine Comedy",
    "The Brothers Karamazov",  "Anna Karenina",  "Madame Bovary",  "The Iliad",  "Don Quixote",  "One Hundred Years of Solitude",  "The Sound and the Fury",
    "Invisible Man", "Beloved", "Mrs. Dalloway", "Slaughterhouse-Five", "The Sun Also Rises",
    "Middlemarch"]


# V is our csv file represented as a matrix to mimic: V = W X H
V = dataset

# to make it easier to see, I made our csv file into a pandas dataframe
df = pd.DataFrame(dataset)
df

,Romance,Fantasy,Historical Fiction,Horror,Humor,Literature,Mystery/Thriller,Science Fiction,Fast paced,Medium paced,Slow paced,Meandering,Engaging
The Great Gatsby,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.420,0.561,0.798,0.050,0.846
To Kill a Mockingbird,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.597,0.727,0.906,0.042,0.045
1984,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.071,0.496,0.019,0.364,0.910
Pride and Prejudice,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.231,0.416,0.544,0.110,0.561
Moby-Dick,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.897,0.973,0.559,0.609,0.822
War and Peace,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.262,0.170,0.325,0.456,0.569
The Catcher in the Rye,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.038,0.164,0.553,0.663,0.098
The Hobbit,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.614,0.459,0.883,0.788,0.804
Ulysses,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.461,0.114,0.837,0.768,0.076
The Odyssey,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.455,0.146,0.543,0.306,0.493


In [6]:
# to estimate the possible shared "topics", I've created a list of 5 topics
topics = ['Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5']

# here we're able to define the model using the NMF package which we'll use to decompose W and H
model = NMF(n_components=5, init='random', random_state=0) 

In [7]:
# creating W from V by taking in the rows of V and the columns from topics
W = model.fit_transform(V)

# creating the other matrix H based on the topics and columns of V
H = pd.DataFrame(model.components_)

# Renaming the indices of H to reflect the topics
H.index = topics

# creating a new numpy array W_df to reflect the dataframe, not the model matrix
# !!!!!!!!!! I rounded to two decimal places just so I can see the data better right now !!!!!!!!!!
# also ensures the columns of W match the rows of H
W_df = pd.DataFrame(np.round(W, 2), columns = H.index)
W_df

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
0,1.09,0.00,0.09,0.02,0.16
1,0.90,0.22,0.84,0.00,0.00
2,0.75,0.00,0.00,0.57,1.03
3,0.60,0.40,0.38,0.00,0.83
4,0.17,0.71,0.34,0.72,0.73
5,0.00,0.51,0.00,0.59,0.90
6,0.00,0.84,0.28,0.03,0.75
7,0.45,0.52,0.11,0.05,0.21
8,0.08,0.00,0.47,0.00,0.86
9,0.76,0.88,0.00,0.58,0.00


In [8]:
# basically repeating everything from W but respectively for H
H.index = topics
H.columns = V.columns
H_df = pd.DataFrame(np.round(H,2))
H_df

,Romance,Fantasy,Historical Fiction,Horror,Humor,Literature,Mystery/Thriller,Science Fiction,Fast paced,Medium paced,Slow paced,Meandering,Engaging
Topic 1,0.00,0.23,0.00,0.00,1.10,0.00,0.00,0.00,0.18,0.14,0.34,0.19,0.40
Topic 2,0.00,0.97,0.00,0.00,0.00,0.00,0.69,1.08,0.21,0.17,0.43,0.31,0.40
Topic 3,1.38,0.08,1.16,0.00,0.04,0.00,0.08,0.00,0.45,0.31,0.39,0.00,0.00
Topic 4,0.00,0.00,0.27,1.61,0.00,0.00,0.04,0.08,0.28,0.20,0.00,0.17,0.00
Topic 5,0.00,0.00,0.00,0.00,0.00,1.08,0.44,0.00,0.02,0.27,0.23,0.37,0.36


In [9]:
# initializing softmax so that every number is converted to a probability
softmax = keras.layers.Softmax()

# actually applying NMF again through matrix-multiplication to reconstruct the approximation of V
reconstructed = softmax(W @ H)

# rounding decimals again to 5 for easier reading
reconstructed = pd.DataFrame(np.round(reconstructed,5), columns=V.columns)
reconstructed.index = V.index
reconstructed.index.name = "Title"
reconstructed

,Romance,Fantasy,Historical Fiction,Horror,Humor,Literature,Mystery/Thriller,Science Fiction,Fast paced,Medium paced,Slow paced,Meandering,Engaging
Title,,,,,,,,,,,,,
The Great Gatsby,0.06177,0.07156,0.06089,0.05654,0.18411,0.06510,0.05933,0.05508,0.06996,0.06866,0.08507,0.07218,0.08974
To Kill a Mockingbird,0.13872,0.07093,0.11535,0.04380,0.12146,0.04349,0.05403,0.05523,0.07801,0.06635,0.08931,0.05548,0.06784
1984,0.04512,0.05356,0.05243,0.11237,0.10241,0.13573,0.07201,0.04692,0.06167,0.07360,0.07303,0.08357,0.08759
Pride and Prejudice,0.07430,0.07704,0.06816,0.04404,0.08649,0.10701,0.08569,0.06805,0.06412,0.07170,0.08911,0.07600,0.08828
Moby-Dick,0.06175,0.08178,0.06940,0.12266,0.04675,0.08441,0.09132,0.08764,0.06663,0.06947,0.07402,0.07323,0.07094
War and Peace,0.04577,0.07471,0.05346,0.11726,0.04562,0.12053,0.09836,0.08265,0.06101,0.07135,0.06954,0.08234,0.07740
The Catcher in the Rye,0.06480,0.10083,0.06117,0.04572,0.04421,0.09807,0.11062,0.10861,0.06044,0.06764,0.08258,0.07523,0.08008
The Hobbit,0.06215,0.09968,0.06144,0.05787,0.08811,0.06750,0.08525,0.09475,0.06912,0.06890,0.08509,0.07501,0.08513
Ulysses,0.10139,0.05594,0.09116,0.05302,0.05885,0.13370,0.07992,0.05302,0.06742,0.07782,0.07921,0.07396,0.07460


In [10]:
# Because the input from the user will be a booke title, we need it to be able to be read as a number for the model
le = LabelEncoder()
# reconstructed["Title"] = le.fit_transform(reconstructed.index)
# reconstructed["Title"] = reconstructed["Title"].astype('float32')
recon_dict = {name: reconstructed[name].values for name in reconstructed.columns}

In [11]:
# re-initializing the pandas dataframe as a tensor dataset
data = tf.data.Dataset.from_tensor_slices(recon_dict)

# just checking that "Title" is being read as a float:
for key, value in recon_dict.items():
    print(f"{key}: dtype={value.dtype}")

Romance: dtype=float32
Fantasy: dtype=float32
Historical Fiction: dtype=float32
Horror: dtype=float32
Humor: dtype=float32
Literature: dtype=float32
Mystery/Thriller: dtype=float32
Science Fiction: dtype=float32
Fast paced: dtype=float32
Medium paced: dtype=float32
Slow paced: dtype=float32
Meandering: dtype=float32
Engaging: dtype=float32


In [12]:
# shuffle data to reduce bias
data = data.shuffle(buffer_size = len(data), reshuffle_each_iteration=False)

# initialize the proportion of the data that we want to use for training and validating
train_size = int(0.7*len(data))
val_size   = int(0.1*len(data))

train = data.take(train_size).batch(5)
val   = data.skip(train_size).take(val_size).batch(5)
test  = data.skip(train_size + val_size).batch(5)

len(train), len(val), len(test)

(4, 1, 2)

## Standardizing inputs

In [13]:
# since the inputs of titles may be inputted differently, want to standardize it, but I don't think I called this anywhere...
def standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    no_punctuation = tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation),'')
    return no_punctuation



# again want to describe the input but earlier I ensured that "Title" was a float but now I'm pretty sure the model's
# expecting a string 
book_input = keras.Input(
    shape = (1,),
    name = "Title",
    dtype = "string"
)

## Training data

In [14]:
# creating the dense layers: not sure if 
main = layers.Dense(64, activation='relu')(book_input)
main = layers.Dropout(0.2)(main)
main = layers.Dense(32, activation='relu')(main)
output = layers.Dense(5, name = "Title_Output")(main) 

model = keras.Model(
    inputs = book_input,
    outputs = output
)

In [15]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Title (InputLayer)                   │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Title_Output (Dense)                 │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,373 (9.27 KB)

 Trainable params: 2,373 (9.27 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# compiling the model using SGD because this or Least Squares would be the best optimizers I think..
model.compile(optimizer = "SGD",
              loss = losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
)

# prevent overfitting
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [17]:

# okay I think I figured it out the reason for the error:
# book_input is expecting the intput to be of a tensor type, but history here is exoecting I think a string "Title"

history = model.fit(train,
                    validation_data=val,
                    epochs = 5,
                    callbacks=[callback],
                    verbose = True)

Epoch 1/5


ValueError: Missing data for input "Title". You passed a data dictionary with keys ['Romance', 'Fantasy', 'Historical Fiction', 'Horror', 'Humor', 'Literature', 'Mystery/Thriller', 'Science Fiction', 'Fast paced', 'Medium paced', 'Slow paced', 'Meandering', 'Engaging']. Expected the following keys: ['Title']